- Câu hỏi: Sinh viên nên thuê trọ vào tháng nào nhất ở một quận cụ thể?  
- Hướng trả lời:
  - Chỉ số x = giá phòng trọ/1m2/1 tháng
  - Tháng nên thuê phòng trọ là tháng có chỉ số x trung bình nhỏ nhất (tính từ năm 2018 đến 2022).
- Nếu trả lời được câu hỏi, thì sinh viên có thể biết tháng nào nên bắt đầu thuê trọ để tiết kiệm chi phí.
- Nguồn cảm hứng xuất phát từ câu hỏi của bản thân khi đi thuê trọ, muốn tiết kiệm tiền. Nếu thuê vào tháng rẻ nhất và ký hợp đồng 1 năm thì em sẽ tiết kiệm được nhiều tiền.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
import seaborn as sns
from datetime import datetime 
import import_ipynb 
import re

In [2]:
df = pd.read_csv('clean_data.csv')
df.head()

,Id,Title,Price(Triệu/tháng),Square(m²),District,City,Date
0,478324,Cho thuê phòng để ở tại 108/64A1 Trần Quang Di...,3.5,18.0,Quận 3,Hồ Chí Minh,20/06/2020
1,728993,Phòng vừa xây xong thoáng mát gần các trường Đ...,3.3,25.0,Quận 7,Hồ Chí Minh,17/10/2018
2,781296,Cần share lại gấp phòng trọ mới thuê tại đường...,3.0,15.0,Quận 10,Hồ Chí Minh,05/10/2018
3,781448,Chính chủ cho thuê căn hộ dịch vụ gần sân bay ...,7.0,27.0,Tân Bình,Hồ Chí Minh,06/10/2018
4,789361,"Phòng cho thuê giá rẻ, Quận Gò Vấp gần chung c...",1.7,12.0,Gò Vấp,Hồ Chí Minh,09/10/2018


In [3]:
def title_clean(feature, df):
    df[feature] = df[feature].str.lower()
    index_lst = []
    e = 'phòng trọ'
    for i in range(df.shape[0]):
        if re.findall(e, df.Title[i]) == []:
            index_lst.append(i)
    df = df.drop(index_lst).reset_index(drop = True)
    return df

In [4]:
# Lọc ra bộ dữ liệu chỉ gồm phòng trọ
df = title_clean('Title', df)

# Xóa những cột mà Price = null và Square = null
df = df[df["Price(Triệu/tháng)"].notna()]
df = df[df["Square(m²)"].notna()]
df.head()

,Id,Title,Price(Triệu/tháng),Square(m²),District,City,Date
0,781296,cần share lại gấp phòng trọ mới thuê tại đường...,3.0,15.0,Quận 10,Hồ Chí Minh,05/10/2018
1,834229,cho thuê phòng trọ quận bình thạnh gần ngã tư ...,1.5,16.0,Bình Thạnh,Hồ Chí Minh,11/10/2018
2,842926,"phòng trọ cách mạng tháng 8, quận 3",4.0,20.0,Quận 3,Hồ Chí Minh,01/10/2018
3,843905,"cho thuê phòng trọ gần chợ hạnh thông tây, đườ...",3.0,20.0,Gò Vấp,Hồ Chí Minh,01/10/2018
4,845417,"phòng trọ ung văn khiêm, quận bình thạnh, dt 25m2",4.5,25.0,Bình Thạnh,Hồ Chí Minh,02/10/2018


In [5]:
# Thêm cột Month, Year
#df['Year'] = df['Date'].str.slice(0,4)
#df['Month'] = df['Date'].str.slice(5,7)
df["Date"] = pd.to_datetime(df["Date"])
year = df['Date'].array
df['Year'] = year[:].year
month = df['Date'].array
df['Month'] = month[:].month

In [6]:
# Thêm cột chỉ số x 
df["X"] = df["Price(Triệu/tháng)"]/df["Square(m²)"]
df = df[["X", "Price(Triệu/tháng)", "Square(m²)", "District", "Month", "Year"]]
df[df["District"] == "Bình Chánh"]
df.rename(columns={'Price(Triệu/tháng)':'Price'},inplace = True)

In [9]:
"""       ** BẢNG DATA BAN ĐẦU **
 + District    Year     Month   Price    ...
 +  A          2018       5      0.1     ...
 +  A          2018       8      0.2     ...
 +  A          2019       5      0.8     ... 
 +  A          2020       8      0.1     ... 
 +  A          2021       8      0.1     ...

        ** BẢNG KẾT QUẢ **
 + District     Month    Price   ...  (không có cột Year)
 + A              5       0.9    
 + A              8       0.4    ... (Giả sử tính sum())
"""
result = df.groupby(['District', 'Month']).mean() # tính trung bình chỉ số X của mỗi quận, trên từng tháng, tính từ năm 2018 - 2022
result

X  Price  Square(m²)     Year
District   Month                                      
Bình Chánh 2      0.066667  2.000      30.000  2020.00
           4      0.080000  2.000      25.000  2021.00
           7      0.014000  1.400     100.000  2022.00
           10     0.066667  2.000      30.000  2020.00
Bình Thạnh 1      0.179136  3.975      21.750  2020.00
...                    ...    ...         ...      ...
Tân Phú    6      0.060227  2.750      41.000  2020.50
           8      0.138889  3.500      25.500  2019.00
           10     0.125361  2.905      21.875  2019.25
           11     0.115964  3.075      26.750  2019.75
           12     0.121667  3.050      25.000  2019.50

[183 rows x 4 columns]

In [10]:
result.loc["Bình Chánh", :]["X"].idxmin()

7

In [11]:
# Lấy chỉ số X của từng tháng
city_list = df["District"].unique()
city_list
for city in city_list:
    print("Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận %s: Tháng"%city,  
    pd.to_numeric(result.loc[city, :]["X"].idxmin()))


Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Quận 10: Tháng 2
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Bình Thạnh: Tháng 3
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Quận 3: Tháng 5
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Gò Vấp: Tháng 10
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Tân Bình: Tháng 12
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Quận 7: Tháng 2
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Bình Tân: Tháng 8
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Tân Phú: Tháng 6
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Quận 2: Tháng 10
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Hóc Môn: Tháng 10
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Phú Nhuận: Tháng 9
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Quận 1: Tháng 7
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay c